In [1]:
from collections import namedtuple
import os
os.chdir("..")
os.getcwd()


'd:\\ineuron\\DL_project\\CNN_project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

#this will act similar namedtuple
@dataclass(frozen=True)
class BaseModelPreparationConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path 
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
  

In [3]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories

In [4]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_base_model_perparation_config(self) -> BaseModelPreparationConfig :
        config = self.config.base_model_preparation

        create_directories([config.root_dir])

        base_model_prepare_config = BaseModelPreparationConfig(
            root_dir = Path(config.root_dir),
            base_model_path= Path(config.base_model_path),
            updated_base_model_path= Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return base_model_prepare_config


In [10]:
#BaseModelPrepareComponent
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class BaseModelPreparation:
    def __init__(self,config:BaseModelPreparationConfig) -> None:
        self.config = config

    #get the base model
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            #they are part of parameters
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
            )
        """
                include_top means whether to include the ANN part of the model or not
        """
        print("base model type:",type(self.model))
        #base_model_path = self.config.base_model_path
        self.save_model(path=self.config.base_model_path , model = self.model)

    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        """
        model: the model which we are using
        classes: output classes(Cat,Dog)
        freeze_all : Freezes weights of all the layers in CNN part.
        freeze_till: freeze weights uptill the layer given in the CNN part.
        learning_rate
        """

        #CNN part
        if (freeze_all):
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        # Ann part
    
        flattern_in = tf.keras.layers.Flatten()(model.output) #passing the model output though a flattern layer method
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flattern_in)  #passing the flattern output though a dense layer
        #the above approch is known as fuctional approch

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    #update the base model
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )
        print("updated model type:",type(self.full_model))
        self.save_model(path = self.config.updated_base_model_path,model = self.full_model)

    @staticmethod
    def save_model(path:Path , model :tf.keras.Model):
        model.save(path)

    

In [12]:
try:
    config = ConfigrationManger()
    base_model_preration_config = config.get_base_model_perparation_config()
    base_model_prepare = BaseModelPreparation(config=base_model_preration_config)
    base_model_prepare.get_base_model()
    base_model_prepare.update_base_model()
except Exception as e:
    raise e

base model type: <class 'keras.engine.functional.Functional'>
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                               

In [10]:
#Experiment on functional approch

class Power:
    def __init__(self,x) -> None:
        self.x = x

    def __call__(self,factor):
        return self.x**factor

    def multi(self):
        return self.x**2

power = Power(3)(12)  #functioal approch
print(power)
power1 = Power(5)
power1.multi()

print(tf.__version__)


531441
2.11.0


list